In [1]:
import numpy as np
import mountain


adad c:\Users\somme\OneDrive - University of Bergen\Uib\Master\Inf266\Reinforcement-learning\assignment2


Run policy evaluation straight left to right

In [ ]:

def policy_evaluation_straight(env, gamma = 1, j = 0):
    grid = env.get_map()
    rows, cols = grid.shape

    V = np.zeros((rows))

    for i in range(cols):
        state = (i, j)
        reward = env.get_reward(state, "forward")
        next_state = env.next_state(state, "forward")
        
        V[i] = reward + gamma * V[next_state[0]]
    
    return V


